###### Content under Creative Commons Attribution license CC-BY 4.0, code under MIT license (c)2014 L.A. Barba, G.F. Forsyth, B. Knaepen

# Let's head in the right direction!

In the previous notebook we applied the Jacobi method to the resolution of the Poisson equation and how several other methods and techniques can speed up the iterative resolution of elliptic PDEs. However, there is still room for improvement. Before looking at the multigrid technique, let us consider the very popular conjugate gradient (CG) method and apply it to the Poisson equation problem that we solved in Lesson 2. The CG method is either used on its own, or in conjunction with multigrid where the latter is used as a preconditioner (more on that in lesson XX).

## Example problem

We again consider the Poisson equation

\begin{equation}
\nabla^2 p = -2\left(\frac{\pi}{2}\right)^2\sin\left( \frac{\pi x}{L} \right) \cos\left(\frac{\pi y}{L}\right)
\end{equation}

in the domain 

$$\left\lbrace \begin{align*}
0 &\leq x\leq 1  \\
-0.5 &\leq y \leq 0.5 
\end{align*} \right.$$

with boundary conditions 

$$p=0 \text{ at } \left\lbrace 
\begin{align*}
x&=0\\
y&=0\\
y&=-0.5\\
y&=0.5
\end{align*} \right.$$

We will solve this equation by assuming an initial state of $p=0$ everywhere, and applying boundary conditions to relax in pseudo-time.

## A bit of theory

The CG method aims at solving elliptic PDEs by iterating the unknown and having it converge, up to a given accuracy, to the solution dictated by the boundary conditions (it is a member of the so-called Krylov subspace methods, but this should not scare you...).

Recall that in its discretized form, the Poisson equation reads,

$$\frac{p_{i+1,j}^{n}-2p_{i,j}^{n}+p_{i-1,j}^{n}}{\Delta x^2}+\frac{p_{i,j+1}^{n}-2 p_{i,j}^{n}+p_{i,j-1}^{n}}{\Delta y^2}=b_{i,j}^{n}$$

You may not have immediately noticed, but you certainly manipulated equations of this kind in the past. Indeed, the left hand side represents a linear combination of the values of $p$ at several grid points and this linear combination has to be equal to the value of the source term on the right hand side.

Now imagine you gather the values $p_{i,j}$ of $p$ at all grid points into a big vector ${\bf p}$ and you do the same for $b$ using the same ordering. Both vectors ${\bf p}$ and ${\bf b}$ contain $N=nx*ny$ values and thus belong to $\mathbf{R}^N$. The linear relationship we just described means that the discretised Poisson equation can be symbolically written as,

\begin{equation}
A{\bf p}={\bf b},
\end{equation}

where $A$ is a $N\times N$ matrix. This is nothing other than a linear system of equations in matrix form. Although we will not directly use this form in the actual CG algorithm below, it is useful to examine the problem this way to understand how the method works.

The starting point is to define the iteration of the potential $p$ as,

\begin{equation}
{\bf p}^{n+1}={\bf p}^n + \alpha {\bf d}^n
\end{equation}

At iteration $n+1$, ${\bf p}^n$ is modified by taking a step in the direction of ${\bf d}^n$. The idea is to start with a guess ${\bf p}^0$ and to march towards the solution by making jumps along the direction vectors ${\bf d}^n$.  $\alpha$ is a scalar that determines how big a jump to take at each iteration.  Now we need to carefully choose the direction vectors and the size of the jumps.

## The method of steepest descent

Before considering the more complex CG algorithm, it is helpful to introduce a simpler approach called the method of steepest descent. At iteration $0$, we choose an inital guess. Unless we are immensely lucky, it will not satisfy the Poisson equation and we will have,

\begin{equation}
{\bf b}-A{\bf p}^0={\bf r}^0\ne {\bf 0}
\end{equation}

The vector ${\bf r}^0$ is called the initial residual and measures how far we are from satisfying the linear system. In fact, we can define such a residual vector at each iteration,

\begin{equation}
{\bf r}^n={\bf b}-A{\bf p}^n
\end{equation}

and eventually monitor how this vector tends to zero as we iterate.

In the method of steepest descent, two choices are made:

1. the direction vectors are chosen to be the residuals ${\bf d}^n = {\bf r}^n$.
2. the length of the jumps are such that the $n+1$ residual is orthogonal to the previous one (because of 1., the direction of jump $n+1$ is orthogonal to the one of jump $n$).

There are good (and, in fact, not complex) reasons to justify these choices and you should read one of the references supplied if you want to understand them. But since we want you to converge to the end of the notebook in a finite time, please accept them for now. 

Condition 2 requires that,

\begin{align}
{\bf r}^{n+1}\cdot {\bf r}^{n} = 0 \nonumber \\
\Leftrightarrow ({\bf b}-A{\bf p}^{n+1}) \cdot {\bf r}^{n} = 0 \nonumber \\
\Leftrightarrow ({\bf b}-A({\bf p}^{n}+\alpha {\bf r}^n)) \cdot {\bf r}^{n} = 0 \nonumber \\
\Leftrightarrow ({\bf r}^n+\alpha A{\bf r}^n) \cdot {\bf r}^{n} = 0 \nonumber \\
\alpha = \frac{{\bf r}^n \cdot {\bf r}^n}{A{\bf r}^n \cdot {\bf r}^n}.
\end{align}

We are now ready to test this algorithm.

To begin, let's import libraries, setup our mesh and import some helper functions as in the previous lesson:

In [ ]:
import numpy
from math import pi
%matplotlib inline

In [ ]:
# Parameters
nx = 41
ny = 41
xmin = 0
xmax = 1
ymin = -0.5
ymax = 0.5


l2_target = 1e-10

In [ ]:
from laplace_helper import plot2D, L2_rel_error
from cg_helper import poisson_2d, initialization, p_analytical

X, Y, x, y, p_i, b, dx, dy, L = initialization(nx, ny, xmax, xmin, ymax, ymin)

pan = p_analytical(X,Y,L)
plot2D(x,y,pan)

#### Figure 1: Analytical solution of the Poisson problem

Below is our implementation of steepest descent.

In [ ]:
def sd_2d(p, b, dx, dy, l2_target):
    '''Performs sd relaxation
    Assumes Dirichlet boundary conditions p=0
    
    Parameters:
    ----------
    p : 2D array of floats
        Initial guess
    b : 2D array of floats
        Source term
    dx: float
        Mesh spacing in x direction
    dy: float
        Mesh spacing in y direction
    l2_target: float
        Error target
    
    Returns:
    -------
    p: 2D array of float
        Distribution after relaxation
    '''
    r  = numpy.zeros((ny,nx)) # residual
    Ar  = numpy.zeros((ny,nx)) # to store result of matrix multiplication
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    # Iterations
    while l2_norm > l2_target:

        pd = p.copy()
        
        r[1:-1,1:-1] = b[1:-1,1:-1]*dx**2 + 4*pd[1:-1,1:-1] - \
            pd[1:-1,2:] - pd[1:-1,:-2] - pd[2:, 1:-1] - pd[:-2, 1:-1]
        
        Ar[1:-1,1:-1] = -4*r[1:-1,1:-1]+r[1:-1,2:]+r[1:-1,:-2]+\
            r[2:, 1:-1] + r[:-2, 1:-1]

        rho = numpy.sum(r*r)
        sigma = numpy.sum(r*Ar)
        alpha = rho/sigma

        p = pd + alpha*r
        
        # BCs are automatically enforced
        
        l2_norm = L2_rel_error(pd,p)
        iterations += 1
        l2_conv.append(l2_norm)
    
    print('Number of SD iterations: {0:d}'.format(iterations))
    return p, l2_conv     



Let's see how it performs on our example problem.

In [ ]:
p, l2_conv = sd_2d(p_i.copy(), b, dx, dy, l2_target)
L2_rel_error(p,pan)

Not bad! it took only two iterations to reach a solution that meets our steady state check. Although this seems great, the steepest descent algorithm is not great when used with large systems or more complex right hand sides in the Poisson equation (see challenge below). We can do better by choosing different jump directions as explained below.

## The method of conjugate gradients

With steepest descent, we know that two **successive** jumps are orthogonal, but that's about it.  There is nothing to prevent the algorithm from making several jumps in the same (or a similar) direction.  Imagine you wanted to go from the intersection of the 5th Avenue and 23rd Street to the intersection of 9th Avenue and 30th Street. Knowing that each segment has the same computational cost (one iteration), would you follow the red path or the green path?

<img src="./figures/jumps.png" width=350>

#### Figure 1. Do you take the red path or the green path?

The method of conjugate gradients is built upon the idea of reducing the number of jumps and makes sure the algorithm never selects the same direction twice. To achieve this, there are two conditions to determine the size of the jumps and the direction vectors:

\begin{equation}
\alpha = \frac{{\bf r}^n \cdot {\bf r}^n}{A{\bf d}^n \cdot {\bf d}^n}
\end{equation}

\begin{equation}
{\bf d}^{n+1}={\bf r}^{n+1}+\beta{\bf d}^{n}
\end{equation}

where $\beta = \frac{{\bf r}^{n+1} \cdot {\bf r}^{n+1}}{{\bf r}^n \cdot {\bf r}^n}$

The search directions are no longer equal to the residuals but are instead a  linear combination of the residual and the previous search direction. It can be shown that this algorithm necessarily converges to the exact solution (up to machine accuracy) in a maximum of $N$ iterations. When one is satisfied with an approximate solution, the number of steps is significantly lower. Again, the derivation of the algorithm is not immensely difficult and can be found in Shewchuk (1994).

Here is a function implementing the CG algorithm:

In [ ]:
def cg_2d(p, b, dx, dy, l2_target):
    '''Performs cg relaxation
    Assumes Dirichlet boundary conditions p=0
    
    Parameters:
    ----------
    p : 2D array of floats
        Initial guess
    b : 2D array of floats
        Source term
    dx: float
        Mesh spacing in x direction
    dy: float
        Mesh spacing in y direction
    l2_target: float
        Error target
    
    Returns:
    -------
    p: 2D array of float
        Distribution after relaxation
    '''
    r  = numpy.zeros((ny,nx)) # residual
    Ad  = numpy.zeros((ny,nx)) # to store result of matrix multiplication 
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    # Step-0 We compute the initial residual and 
    # the first search direction is just this residual
    
    r[1:-1,1:-1] = b[1:-1,1:-1]*dx**2 + 4*p[1:-1,1:-1] - \
        p[1:-1,2:] - p[1:-1,:-2] - p[2:, 1:-1] - p[:-2, 1:-1]
    d = r
    rho = numpy.sum(r*r)
    Ad[1:-1,1:-1] = -4*d[1:-1,1:-1]+d[1:-1,2:]+d[1:-1,:-2]+\
        d[2:, 1:-1] + d[:-2, 1:-1]
    sigma = numpy.sum(d*Ad)
    
    # Iterations
    while l2_norm > l2_target:

        pd = p.copy()
        rd = r.copy()
        dd = d.copy()
        
        alpha = rho/sigma

        p = pd + alpha*dd
        r = rd - alpha*Ad
        
        rhop1 = numpy.sum(r*r)
        beta = rhop1 / rho
        rho = rhop1
        
        d = r + beta*dd
        Ad[1:-1,1:-1] = -4*d[1:-1,1:-1] + d[1:-1,2:] + d[1:-1,:-2] + \
            d[2:, 1:-1] + d[:-2, 1:-1]
        sigma = numpy.sum(d*Ad)
        
        # BCs are automatically enforced
        
        l2_norm = L2_rel_error(pd,p)
        iterations += 1
        l2_conv.append(l2_norm)
    
    print('Number of CG iterations: {0:d}'.format(iterations))
    return p, l2_conv     


In [ ]:
p, l2_conv = cg_2d(p_i.copy(), b, dx, dy, l2_target)
L2_rel_error(p,pan)

Let's compare to the number of iterations needed for the Jacobi iteration:

In [ ]:
p, l2_conv = poisson_2d(p_i.copy(), b, dx, dy, l2_target)

## References

Shewchuk, J. (1994). An Introduction to the Conjugate Gradient Method Without the Agonizing Pain. URL: http://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf

In [ ]:
from IPython.core.display import HTML
css_file = '../../styles/numericalmoocstyle.css'
HTML(open(css_file, "r").read())